# UT Dataset Exploration

This notebook explores the Utah (UT) dataset to understand household counts, income distribution, and demographic characteristics.

In [1]:
from policyengine_us import Microsimulation
import pandas as pd
import numpy as np

UT_DATASET = "hf://policyengine/policyengine-us-data/states/UT.h5"

In [2]:
# Load UT dataset
sim = Microsimulation(dataset=UT_DATASET)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


UT.h5:   0%|          | 0.00/39.1M [00:00<?, ?B/s]

In [3]:
# Check dataset size
household_weight = sim.calculate("household_weight", period=2025)
household_count = sim.calculate("household_count", period=2025, map_to="household")
person_count = sim.calculate("person_count", period=2025, map_to="household")

print(f"Number of households in dataset: {len(household_weight):,}")
print(f"Household count (weighted): {household_count.sum():,.0f}")
print(f"Person count (weighted): {person_count.sum():,.0f}")

Number of households in dataset: 17,648
Household count (weighted): 1,015,265
Person count (weighted): 3,467,711


In [4]:
# Check household income distribution
agi = sim.calculate("adjusted_gross_income", period=2025, map_to="household")
print(f"Income distribution:")
print(f"  Median AGI: ${agi.median():,.0f}")
print(f"  75th percentile: ${agi.quantile(0.75):,.0f}")
print(f"  90th percentile: ${agi.quantile(0.90):,.0f}")
print(f"  95th percentile: ${agi.quantile(0.95):,.0f}")
print(f"  Max AGI: ${agi.max():,.0f}")

Income distribution:
  Median AGI: $104,469
  75th percentile: $214,444
  90th percentile: $432,171
  95th percentile: $541,148
  Max AGI: $3,229,514


In [5]:
# Check households with children
is_child = sim.calculate("is_child", period=2025, map_to="person")
household_id = sim.calculate("household_id", period=2025, map_to="person")
household_weight = sim.calculate("household_weight", period=2025, map_to="person")

# Create DataFrame
df_households = pd.DataFrame({
    'household_id': household_id,
    'is_child': is_child,
    'household_weight': household_weight
})

# Count children per household
children_per_household = df_households.groupby('household_id').agg({
    'is_child': 'sum',
    'household_weight': 'first'
}).reset_index()

# Calculate weighted household counts
total_households_with_children = children_per_household[children_per_household['is_child'] > 0]['household_weight'].sum()
households_with_1_child = children_per_household[children_per_household['is_child'] == 1]['household_weight'].sum()
households_with_2_children = children_per_household[children_per_household['is_child'] == 2]['household_weight'].sum()
households_with_3plus_children = children_per_household[children_per_household['is_child'] >= 3]['household_weight'].sum()

print(f"\nHouseholds with children (weighted):")
print(f"  Total households with children: {total_households_with_children:,.0f}")
print(f"  Households with 1 child: {households_with_1_child:,.0f}")
print(f"  Households with 2 children: {households_with_2_children:,.0f}")
print(f"  Households with 3+ children: {households_with_3plus_children:,.0f}")


Households with children (weighted):
  Total households with children: 452,300
  Households with 1 child: 167,413
  Households with 2 children: 159,249
  Households with 3+ children: 125,638


In [10]:
# Check children by age groups
df = pd.DataFrame({
    "household_id": sim.calculate("household_id", map_to="person"),
    "tax_unit_id": sim.calculate("tax_unit_id", map_to="person"),
    "person_id": sim.calculate("person_id", map_to="person"),
    "age": sim.calculate("age", map_to="person"),
    "person_weight": sim.calculate("person_weight", map_to="person")
})

# Filter for children and apply weights
children_under_18_df = df[df['age'] < 18]
children_under_6_df = df[df['age'] < 6]
children_under_3_df = df[df['age'] < 3]

# Calculate weighted totals
total_children = children_under_18_df['person_weight'].sum()
children_under_6 = children_under_6_df['person_weight'].sum()
children_under_3 = children_under_3_df['person_weight'].sum()

print(f"\nChildren by age:")
print(f"  Total children under 18: {total_children:,.0f}")
print(f"  Children under 6: {children_under_6:,.0f}")
print(f"  Children under 3: {children_under_3:,.0f}")


Children by age:
  Total children under 18: 931,582
  Children under 6: 276,080
  Children under 3: 120,763


In [ ]:
# Create weighted summary table
weighted_summary_data = {
    'Metric': [
        'Household count (weighted)',
        'Person count (weighted)',
        'Median AGI',
        '75th percentile AGI',
        '90th percentile AGI',
        '95th percentile AGI',
        'Max AGI',
        'Total households with children',
        'Households with 1 child',
        'Households with 2 children',
        'Households with 3+ children',
        'Total children under 18',
        'Children under 6',
        'Children under 3'
    ],
    'Value': [
        f"{household_count.sum():,.0f}",
        f"{person_count.sum():,.0f}",
        f"${agi.median():,.0f}",
        f"${agi.quantile(0.75):,.0f}",
        f"${agi.quantile(0.90):,.0f}",
        f"${agi.quantile(0.95):,.0f}",
        f"${agi.max():,.0f}",
        f"{total_households_with_children:,.0f}",
        f"{households_with_1_child:,.0f}",
        f"{households_with_2_children:,.0f}",
        f"{households_with_3plus_children:,.0f}",
        f"{total_children:,.0f}",
        f"{children_under_6:,.0f}",
        f"{children_under_3:,.0f}"
    ]
}

weighted_df = pd.DataFrame(weighted_summary_data)

print("\n" + "="*60)
print("UT DATASET SUMMARY - WEIGHTED (Population Estimates)")
print("="*60)
print(weighted_df.to_string(index=False))
print("="*60)

# Save table
weighted_df.to_csv('ut_dataset_summary_weighted.csv', index=False)
print("\nSummary saved to: ut_dataset_summary_weighted.csv")


UT DATASET SUMMARY - WEIGHTED (Population Estimates)
                        Metric      Value
    Household count (weighted)  1,015,265
       Person count (weighted)  3,467,711
                    Median AGI   $104,469
           75th percentile AGI   $214,444
           90th percentile AGI   $432,171
           95th percentile AGI   $541,148
                       Max AGI $3,229,514
Total households with children    452,300
       Households with 1 child    167,413
    Households with 2 children    159,249
   Households with 3+ children    125,638
       Total children under 18    931,582
              Children under 6    276,080
              Children under 3    120,763

Summary saved to: ut_dataset_summary_weighted.csv


: 

In [8]:
# Households with $0 income
agi_hh = np.array(sim.calculate("adjusted_gross_income", period=2025, map_to="household"))
weights = np.array(sim.calculate("household_weight", period=2025))

zero_income_mask = agi_hh == 0
zero_income_count = weights[zero_income_mask].sum()
total_households = weights.sum()

print("\n" + "="*70)
print("HOUSEHOLDS WITH $0 INCOME")
print("="*70)
print(f"Household count: {zero_income_count:,.0f}")
print(f"Percentage of all households: {zero_income_count / total_households * 100:.2f}%")
print("="*70)


HOUSEHOLDS WITH $0 INCOME
Household count: 48,395
Percentage of all households: 4.77%


In [9]:
# Household counts by income brackets
income_brackets = [
    (0, 10000, "$0-$10k"),
    (10000, 20000, "$10k-$20k"),
    (20000, 30000, "$20k-$30k"),
    (30000, 40000, "$30k-$40k"),
    (40000, 50000, "$40k-$50k"),
    (50000, 60000, "$50k-$60k")
]

bracket_data = []
for lower, upper, label in income_brackets:
    mask = (agi_hh >= lower) & (agi_hh < upper)
    count = weights[mask].sum()
    pct_of_total = (count / total_households) * 100
    
    bracket_data.append({
        "Income Bracket": label,
        "Households": f"{count:,.0f}",
        "% of All Households": f"{pct_of_total:.2f}%"
    })

income_df = pd.DataFrame(bracket_data)

print("\n" + "="*70)
print("HOUSEHOLD COUNTS BY INCOME BRACKET")
print("="*70)
print(income_df.to_string(index=False))
print("="*70)

# Total in $0-$60k range
total_in_range = sum([weights[(agi_hh >= lower) & (agi_hh < upper)].sum() for lower, upper, _ in income_brackets])
print(f"\nTotal households in $0-$60k range: {total_in_range:,.0f}")
print(f"Percentage of all households in $0-$60k range: {total_in_range / total_households * 100:.2f}%")


HOUSEHOLD COUNTS BY INCOME BRACKET
Income Bracket Households % of All Households
       $0-$10k    126,445              12.45%
     $10k-$20k     15,102               1.49%
     $20k-$30k     21,610               2.13%
     $30k-$40k     71,543               7.05%
     $40k-$50k     62,599               6.17%
     $50k-$60k     40,115               3.95%

Total households in $0-$60k range: 337,414
Percentage of all households in $0-$60k range: 33.23%
